In [ ]:
import subprocess
import configparser
import os
import sys
import shutil
from sync_git_lib.sync_git_lib import *

#argv_list = ["./from_sandbox_python_to_sandbox_python2/setting.ini", "src_user", "src_pass", "dst_usr", "dst_pass"]
argv_list = ["./from_sandbox_python_to_sandbox_python2/setting.ini", "aaa", "bbb", "ccc", "ddd"]

In [ ]:
start_dir = os.path.abspath(".")
print("DEBUG: start dir = " + start_dir)

In [ ]:
# =========================
# 設定ファイルの読み込み
# =========================
setting_file_path = os.path.abspath(argv_list[0]) # コマンドライン引数にする
print("DEBUG: setting_file_path = " + setting_file_path)

setting_file = configparser.ConfigParser()
setting_file.read(setting_file_path, "UTF-8")

In [ ]:
src_rep_web_url = setting_file.get("SRC_REPOSITORY", "web_url")
print("DEBUG: src_rep_web_url = " + src_rep_web_url)

src_rep_clone_url = setting_file.get("SRC_REPOSITORY", "clone_url")
print("DEBUG: src_rep_clone_url = " + src_rep_clone_url)

src_rep_branch = setting_file.get("SRC_REPOSITORY", "branch")
print("DEBUG: src_rep_branch = " + src_rep_branch)

#src_rep_user = setting_file.get("SRC_REPOSITORY", "user") # コマンドライン引数にする
src_rep_user = argv_list[1]
print("DEBUG: src_rep_user = " + src_rep_user)

#src_rep_pass = setting_file.get("SRC_REPOSITORY", "pass") # コマンドライン引数にする
src_rep_pass = argv_list[2]
print("DEBUG: src_rep_pass = " + src_rep_pass)

src_rep_clone_url_with_user_and_pass = make_clone_url_with_user_and_pass(src_rep_clone_url, src_rep_user, src_rep_pass)
print("DEBUG: src_rep_clone_url_with_user_and_pass = " + src_rep_clone_url_with_user_and_pass)

src_rep_dir = setting_file.get("SRC_REPOSITORY", "dir")
print("DEBUG: src_rep_dir = " + src_rep_dir)

src_rep_label = setting_file.get("SRC_REPOSITORY", "label")
print("DEBUG: src_rep_label = " + src_rep_label)

the_latest_sync_hash = setting_file.get("SRC_REPOSITORY", "the_latest_sync_hash")
print("DEBUG: the_latest_sync_hash = " + the_latest_sync_hash)

In [ ]:
dst_rep_web_url = setting_file.get("DST_REPOSITORY", "web_url")
print("DEBUG: dst_rep_web_url = " + dst_rep_web_url)

dst_rep_clone_url = setting_file.get("DST_REPOSITORY", "clone_url")
print("DEBUG: dst_rep_clone_url = " + dst_rep_clone_url)

dst_rep_branch = setting_file.get("DST_REPOSITORY", "branch")
print("DEBUG: dst_rep_branch = " + dst_rep_branch)

#dst_rep_user = setting_file.get("DST_REPOSITORY", "user") # コマンドライン引数にする
dst_rep_user = argv_list[3]
print("DEBUG: dst_rep_user = " + dst_rep_user)

#dst_rep_pass = setting_file.get("DST_REPOSITORY", "pass") # コマンドライン引数にする
dst_rep_pass = argv_list[4]
print("DEBUG: dst_rep_pass = " + dst_rep_pass)

dst_rep_clone_url_with_user_and_pass = make_clone_url_with_user_and_pass(dst_rep_clone_url, dst_rep_user, dst_rep_pass)
print("DEBUG: dst_rep_clone_url_with_user_and_pass = " + dst_rep_clone_url_with_user_and_pass)

dst_rep_dir = setting_file.get("DST_REPOSITORY", "dir")
print("DEBUG: dst_rep_dir = " + dst_rep_dir)

dst_rep_label = setting_file.get("DST_REPOSITORY", "label")
print("DEBUG: dst_rep_label = " + dst_rep_label)

In [ ]:
work_dir = os.path.abspath(setting_file.get("TOOL", "work_dir"))
print("DEBUG: work_dir = " + work_dir)

info_label = setting_file.get("TOOL", "info_label")
print("DEBUG: info_label = " + info_label)

tool_label = setting_file.get("TOOL", "tool_label")
print("DEBUG: tool_label = " + tool_label)

In [ ]:
# 既にworkディレクトリがあったら異常終了する
if os.path.isdir(work_dir):
    print("ERROR: word directory, " + work_dir + " already exists.")
    sys.exit(1)

# workディレクト作成 & 移動
os.mkdir(work_dir)
os.chdir(work_dir)
print("DEBUG: current dir = " + os.path.abspath("."))

In [ ]:
# srcリポジトリをclone
clone_repository(src_rep_clone_url_with_user_and_pass, src_rep_dir)

In [ ]:
# dstリポジトリをclone
clone_repository(dst_rep_clone_url_with_user_and_pass, dst_rep_dir)

In [ ]:
# =========================
# 以降はsrcリポジトリ配下での操作
# =========================
os.chdir(src_rep_dir)

In [ ]:
# 設定ファイルで指定されたbranchへ移動
switch_local_branch_on_src_repository(src_rep_branch)

In [ ]:
# HEADのhashを取得
src_rep_head_hash = get_head_hash()
print("DEBUG: src_rep_head_hash=" + src_rep_head_hash + ", the_latest_sync_hash=" + the_latest_sync_hash)

# 最後に同期したhashとHEAD hashを比較して、同じだったら同期の必要はないので終了
if src_rep_head_hash == the_latest_sync_hash:
    print("Info: There is nothing to sync. the_latest_sync_hash = " + the_latest_sync_hash + ", src_rep_head_hash = " + src_rep_head_hash )
    os.chdir(start_dir)
    shutil.rmtree(work_dir)
    sys.exit(0)

In [ ]:
# git log のlistを取得
git_log_list = get_git_log_list(the_latest_sync_hash, src_rep_head_hash)
print("DEBUG: git_log_list = \n" + "\n".join(git_log_list))

In [ ]:
# 無視するgit logを抽出するためのキーワードリストを作る
ignored_keyword_list = [dst_rep_label, info_label]
print("DEBUG: ignored_keyword_list = \n" + "\n".join(ignored_keyword_list))

# patch化するgit logのリストを作る
patched_git_log_list = git_log_list
for ignored_keyword in ignored_keyword_list:
    keyword = tool_label + ignored_keyword
    patched_git_log_list = [git_log for git_log in patched_git_log_list if keyword not in git_log]
print("DEBUG: patched_git_log_list = \n" + "\n".join(patched_git_log_list))

# patch化するhashリストを作る
patch_hash_list = [git_log.split(",")[-1] for git_log in patched_git_log_list]
patch_hash_list = patch_hash_list[::-1] #並び順をリバースする(古い順にするため)
print("DEBUG: patch_hash_list = \n" + "\n".join(patch_hash_list))

In [ ]:
# patch化するdiffファイル名のリストを作る
patch_file_list = []
for index_, hash_ in zip(range(len(patch_hash_list)), patch_hash_list):
    patch_file_list += [("{0:03d}".format(index_) + "_" + hash_ + ".patch")]
print("DEBUG: patch_file_list = \n" + "\n".join(patch_file_list))

In [ ]:
# 実際にpatchファイルを作る
for hash_, file_ in zip(patch_hash_list, patch_file_list):
    make_patch_file(work_dir + "/" + file_, hash_)

In [ ]:
# =========================
# 以降はdstリポジトリ配下での操作
# =========================
os.chdir(work_dir)
os.chdir(dst_rep_dir)
print("DEBUG: current dir = " + os.path.abspath(".") )

In [ ]:
# 設定ファイルで指定されたbranchへ移動
dst_rep_current_branch = switch_local_branch_on_dst_repository(dst_rep_branch)
print("DEBUG: dst_rep_current_branch = " + dst_rep_current_branch)

In [ ]:
# 1つずつ、patchファイルを当ててコミットしていく & 最後にpush
commit_label = tool_label + src_rep_label
for index_, hash_, file_ in zip(range(len(patch_hash_list)), patch_hash_list, patch_file_list):
    git_apply_patch(work_dir + "/" + file_)
    
    git_add_all()
    
    commit_message = '-m "' + commit_label + " " + str(index_+1) + "/" + str(len(patch_hash_list)) + ' merged" '
    commit_message += '-m "" '
    commit_message += '-m "' + src_rep_web_url + "/commit/" + hash_ + '"'
    git_commit(commit_message)
    
git_push(dst_rep_branch)

In [ ]:
# =========================
# 以降はtoolリポジトリ配下での操作
# =========================
os.chdir(start_dir)

In [ ]:
# 設定ファイルの最後に同期したhash値をHEAD hashに更新する
setting_file.set("SRC_REPOSITORY", "the_latest_sync_hash", src_rep_head_hash)

#設定ファイルのbranchを更新
if not dst_rep_branch == dst_rep_current_branch:
    setting_file.set("DST_REPOSITORY", "branch", dst_rep_current_branch)

fileobj = open(setting_file_path, "w")
setting_file.write(fileobj)
fileobj.close()

print("DEBUG: DST_REPOSITORY.the_latest_sync_hash = " + the_latest_sync_hash + " -> " + src_rep_head_hash)
print("DEBUG: DST_REPOSITORY.branch = " + dst_rep_branch + " -> " + dst_rep_current_branch)

In [ ]:
setting_file_path = argv_list[0] #仮対処

# 設定した更新ファイルをコミットする & 最後にpush
git_add(setting_file_path)

commit_message = '-m "' + tool_label + " " + src_rep_label + "->" + dst_rep_label + " merged " + str(len(patch_hash_list)) + ' hash" '
commit_message += '-m "" '
for patch_hash in patch_hash_list[::-1]: # hashを新しい順に並べる
    commit_message += '-m "' + src_rep_web_url + "/commit/" + patch_hash + '" '
git_commit(commit_message)

git_push("")

In [ ]:
# 最後にworkディレクトリを消して終了
shutil.rmtree(work_dir)